In [3]:
#Importing the necessary packages
import pandas as pd
import numpy as np
import glob
import time

### Function to create one dataframe from all individual monthly files

#### This is an old function and is not used for the final dataframe creation. However, this is kept as a reference for functions written later

In [52]:
def create_df():
    
    #Total excel files = 204
    #Individual File shape = (852, 6)
    #Final Dataframe shape = (173808, 8)
    
    column_names = ["row_no", "min_K", "max_K", "mean_K", "geoid", "name"] #List of column names of individual files
    path = "narr_urban_county_data/air.sfc"
    all_files = glob.glob(path + "/*.csv")
    data = pd.DataFrame(columns = column_names) #Defining an empty dataframe
    for filename in all_files:
        temp_df = pd.read_csv(filename, index_col=None, header=0, names=column_names) #Individual files in a temp dataframe
        year = filename[-11:-7]
        month = filename[-6:-4]
        temp_df.insert(6, "year", year) #Adding year and month from filename for further use
        temp_df.insert(7, "month", month)
        temp_df["date"] = pd.to_datetime(temp_df[['year','month']].assign(DAY=1))
        data = data.append(temp_df, ignore_index=True, sort=False) #The temp dataframe is not appended to final df
    idx = pd.date_range('1999-01-01', '2015-12-01', freq='MS') #Specifying the datetime index
    data = data[["name", "date", "year", "month", "max_K", "mean_K", "min_K", "geoid"]] #Re-ordering the columns
    
    return data

## Loading the dataframe

In [53]:
# Temperature Data

start = time.process_time() #To calculate the time taken to read 204 files
data = create_df()   
print("Dataframe created in ", time.process_time() - start, "seconds")
print("Shape of the Dataframe: ", data.shape)
data.head(5)

Dataframe created in  10.638253999999506 seconds
Shape of the Dataframe:  (173808, 8)


,name,date,year,month,max_K,mean_K,min_K,geoid
0,Lancaster,2005-04-01,2005,04,287.734802,287.612427,287.537933,31109
1,Minnehaha,2005-04-01,2005,04,287.369446,286.633260,285.855377,46099
2,Allen,2005-04-01,2005,04,284.916840,284.776489,284.636108,39003
3,Beaver,2005-04-01,2005,04,283.967102,283.803630,283.577515,42007
4,Chatham,2005-04-01,2005,04,288.692871,288.243927,287.893127,37037


In [54]:
#WNV Incedence Rate (IR) data

wnv = pd.read_csv("WNV_NI_NNI_1999to2015_prevalence_incidence_final_20180530.csv", encoding="ISO-8859-1")
print("Shape of the Dataframe: ", wnv.shape)
wnv.head(2)

Shape of the Dataframe:  (3108, 153)


,GEOID10,Select_County,STATEFP10,STATENS,STUSPS,STNAME,COUNTYFP10,COUNTYNS10,CTYNAME,POPESTIMATE1999,...,NIIR_2011_Z,NIIR_2012_Z,NIIR_2013_Z,NIIR_2014_Z,NIIR_2015_Z,NIIR_EST_AVG_Z,NIIR_EST_MED_Z,County_WNV_Class,County_WNV_Class2,filter_$
0,1001,1,1,1779775,AL,Alabama,1,161526,Autauga County,42963,...,-.172668378274114,-.313041876784145,-.237005866057391,-.211466937746909,-.197767338621807,-.230659201920693,-.231569016088473,2,NI-S,0
1,1003,1,1,1779775,AL,Alabama,3,161527,Baldwin County,137555,...,-.172668378274114,.0148439683883608,-.177634573907632,-.211466937746909,-.197767338621807,-.177866829768146,-.186748999098691,4,NI-M-pre12,0


### Adding climate zone column to the dataframe

In [56]:
ansi = pd.read_csv("ANSI.csv") #This file is present in the box
ansi_dict = ansi.set_index('ANSI').state.to_dict()
print(ansi_dict)

{1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC', 12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS', 21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS', 29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY', 37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC', 46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV', 55: 'WI', 56: 'WY', 60: 'AS', 66: 'GU', 69: 'MP', 72: 'PR', 74: 'UM', 78: 'VI'}


In [57]:
print(data.dtypes)
data.head(2)

name              object
date      datetime64[ns]
year              object
month             object
max_K            float64
mean_K           float64
min_K            float64
geoid             object
dtype: object


,name,date,year,month,max_K,mean_K,min_K,geoid
0,Lancaster,2005-04-01,2005,04,287.734802,287.612427,287.537933,31109
1,Minnehaha,2005-04-01,2005,04,287.369446,286.633260,285.855377,46099


In [58]:
data['ansi'] = data['geoid'].astype(str).str[:2]
data['ansi'] = data['ansi'].astype(int)
data['state'] = data['ansi'].map(ansi_dict)
data.head(2)

,name,date,year,month,max_K,mean_K,min_K,geoid,ansi,state
0,Lancaster,2005-04-01,2005,04,287.734802,287.612427,287.537933,31109,31,NE
1,Minnehaha,2005-04-01,2005,04,287.369446,286.633260,285.855377,46099,46,SD


### Adding Climate Zone column
#### Central (c) , East North Central (enc) , Northeast (ne) , Northwest (nw) , South (s) , Southeast (se) , Southwest (sw) , West (w) , West North Central (wnc)

In [59]:
#Creating climate dictionary
#Creating it this way as it is easier to type but this dictionary cannot be used to create climate zone column
climate_dict = {'wnc' : ['MT','NE','ND','SD','WY'],
                'enc' : ['IA','MI','MN','WI'],
                'nw' : ['ID','OR','WA'],
                'ne' : ['CT','DE','ME','MD','MA','NH','NJ','NY','PA','RI','VT'],
                'se' : ['AL','FL','GA','NC','SC','VA'],
                'sw' : ['AZ','CO','NM','UT'],
                'c' : ['IL','IN','KY','MO','OH','TN','WV'],
                's' : ['AR','KS','LA','MS','OK','TX'],
                'w' : ['CA','NV']
               }

#The new dictionary swaps keys and values from above dictionary to create the climate zone column
climate_dict2 = {} 
for key, value in climate_dict.items():
    for items in value:
        climate_dict2[items]=key
        
print(climate_dict2)

{'MT': 'wnc', 'NE': 'wnc', 'ND': 'wnc', 'SD': 'wnc', 'WY': 'wnc', 'IA': 'enc', 'MI': 'enc', 'MN': 'enc', 'WI': 'enc', 'ID': 'nw', 'OR': 'nw', 'WA': 'nw', 'CT': 'ne', 'DE': 'ne', 'ME': 'ne', 'MD': 'ne', 'MA': 'ne', 'NH': 'ne', 'NJ': 'ne', 'NY': 'ne', 'PA': 'ne', 'RI': 'ne', 'VT': 'ne', 'AL': 'se', 'FL': 'se', 'GA': 'se', 'NC': 'se', 'SC': 'se', 'VA': 'se', 'AZ': 'sw', 'CO': 'sw', 'NM': 'sw', 'UT': 'sw', 'IL': 'c', 'IN': 'c', 'KY': 'c', 'MO': 'c', 'OH': 'c', 'TN': 'c', 'WV': 'c', 'AR': 's', 'KS': 's', 'LA': 's', 'MS': 's', 'OK': 's', 'TX': 's', 'CA': 'w', 'NV': 'w'}


In [60]:
data['climate_zone'] = data['state'].map(climate_dict2) #Mapping the values to a dataframe column
data.head(5)

,name,date,year,month,max_K,mean_K,min_K,geoid,ansi,state,climate_zone
0,Lancaster,2005-04-01,2005,04,287.734802,287.612427,287.537933,31109,31,NE,wnc
1,Minnehaha,2005-04-01,2005,04,287.369446,286.633260,285.855377,46099,46,SD,wnc
2,Allen,2005-04-01,2005,04,284.916840,284.776489,284.636108,39003,39,OH,c
3,Beaver,2005-04-01,2005,04,283.967102,283.803630,283.577515,42007,42,PA,ne
4,Chatham,2005-04-01,2005,04,288.692871,288.243927,287.893127,37037,37,NC,se


In [61]:
data.state.unique()

array(['NE', 'SD', 'OH', 'PA', 'NC', 'VA', 'TN', 'LA', 'TX', 'MA', 'FL',
       'OK', 'GA', nan, 'WV', 'NJ', 'OR', 'KY', 'IN', 'UT', 'IL', 'ME',
       'ID', 'NV', 'WI', 'NY', 'MD', 'VT', 'KS', 'DE', 'MI', 'AS', 'ND',
       'MO', 'MS', 'MN', 'SC', 'IA', 'DC', 'NM', 'MT', 'WA', 'RI', 'NH',
       'WY'], dtype=object)

### Average annual incidence of West Nile virus neuroinvasive disease reported to CDC by state, 1999-2018
#### Source: https://www.cdc.gov/westnile/resources/pdfs/data/WNV-Neuro-Incidence-by-State-Map_1999-2018-P.pdf
#### A = (0.01 - 0.24); B = (0.25 - 0.49); C = (0.50 - 0.74); D = (0.75 - 0.99); E = (>=1.00)

In [62]:
wnv_zones = pd.read_csv("Average_annual_incidence_by_state.csv", encoding='ISO-8859-1')
wnv_zones = wnv_zones[["Abbr", "Average_Incidence"]]
wnv_zones = wnv_zones.sort_values(by=['Average_Incidence'])
IR_dict = wnv_zones.set_index("Abbr").Average_Incidence.to_dict()
print(IR_dict)

{'ME': 0.01, 'NH': 0.02, 'NC': 0.03, 'OR': 0.05, 'WA': 0.05, 'WV': 0.05, 'VT': 0.06, 'RI': 0.07, 'FL': 0.08, 'SC': 0.08, 'VA': 0.1, 'NJ': 0.12, 'MA': 0.12, 'CT': 0.15, 'GA': 0.15, 'KY': 0.15, 'DE': 0.16, 'NY': 0.18, 'PA': 0.18, 'TN': 0.19, 'WI': 0.2, 'MD': 0.22, 'AL': 0.25, 'OH': 0.32, 'MN': 0.32, 'NV': 0.32, 'IN': 0.33, 'UT': 0.34, 'MO': 0.39, 'AR': 0.44, 'IA': 0.51, 'CA': 0.53, 'MI': 0.56, 'KS': 0.6, 'IL': 0.65, 'OK': 0.66, 'TX': 0.67, 'ID': 0.72, 'NM': 0.77, 'AZ': 0.85, 'MT': 0.94, 'LA': 1.21, 'MS': 1.31, 'CO': 1.32, 'WY': 1.61, 'NE': 2.14, 'ND': 3.07, 'SD': 3.48}


In [63]:
#The following block of code is creating a dictionary of wnv_zone for all states

wnv_dict = {}
for key, value in IR_dict.items():
    if (value>0) and (value<0.25):
        wnv_dict[key] = "A"
    elif (value>=0.25) and (value<0.50):
        wnv_dict[key] = "B"
    elif (value>=0.50) and (value<0.75):
        wnv_dict[key] = "C"
    elif (value>=0.75) and (value<1.00):
        wnv_dict[key] = "D"
    else:
        wnv_dict[key] = "E"
        
print(wnv_dict)

{'ME': 'A', 'NH': 'A', 'NC': 'A', 'OR': 'A', 'WA': 'A', 'WV': 'A', 'VT': 'A', 'RI': 'A', 'FL': 'A', 'SC': 'A', 'VA': 'A', 'NJ': 'A', 'MA': 'A', 'CT': 'A', 'GA': 'A', 'KY': 'A', 'DE': 'A', 'NY': 'A', 'PA': 'A', 'TN': 'A', 'WI': 'A', 'MD': 'A', 'AL': 'B', 'OH': 'B', 'MN': 'B', 'NV': 'B', 'IN': 'B', 'UT': 'B', 'MO': 'B', 'AR': 'B', 'IA': 'C', 'CA': 'C', 'MI': 'C', 'KS': 'C', 'IL': 'C', 'OK': 'C', 'TX': 'C', 'ID': 'C', 'NM': 'D', 'AZ': 'D', 'MT': 'D', 'LA': 'E', 'MS': 'E', 'CO': 'E', 'WY': 'E', 'NE': 'E', 'ND': 'E', 'SD': 'E'}


In [64]:
data['wnv_zone'] = data['state'].map(wnv_dict) #Mapping the values to a dataframe column
data.head(5)

,name,date,year,month,max_K,mean_K,min_K,geoid,ansi,state,climate_zone,wnv_zone
0,Lancaster,2005-04-01,2005,04,287.734802,287.612427,287.537933,31109,31,NE,wnc,E
1,Minnehaha,2005-04-01,2005,04,287.369446,286.633260,285.855377,46099,46,SD,wnc,E
2,Allen,2005-04-01,2005,04,284.916840,284.776489,284.636108,39003,39,OH,c,B
3,Beaver,2005-04-01,2005,04,283.967102,283.803630,283.577515,42007,42,PA,ne,A
4,Chatham,2005-04-01,2005,04,288.692871,288.243927,287.893127,37037,37,NC,se,A


## Creating the final dataframe

### Adding Temperature, Humidity and Precipitation data

In [67]:
#Understanding the WNV data
wnv.head(2)

,GEOID10,Select_County,STATEFP10,STATENS,STUSPS,STNAME,COUNTYFP10,COUNTYNS10,CTYNAME,POPESTIMATE1999,...,NIIR_2011_Z,NIIR_2012_Z,NIIR_2013_Z,NIIR_2014_Z,NIIR_2015_Z,NIIR_EST_AVG_Z,NIIR_EST_MED_Z,County_WNV_Class,County_WNV_Class2,filter_$
0,1001,1,1,1779775,AL,Alabama,1,161526,Autauga County,42963,...,-.172668378274114,-.313041876784145,-.237005866057391,-.211466937746909,-.197767338621807,-.230659201920693,-.231569016088473,2,NI-S,0
1,1003,1,1,1779775,AL,Alabama,3,161527,Baldwin County,137555,...,-.172668378274114,.0148439683883608,-.177634573907632,-.211466937746909,-.197767338621807,-.177866829768146,-.186748999098691,4,NI-M-pre12,0


In [68]:
print(wnv.shape, "\n")
print(list(wnv.columns))

(3108, 153) 

['GEOID10', 'Select_County', 'STATEFP10', 'STATENS', 'STUSPS', 'STNAME', 'COUNTYFP10', 'COUNTYNS10', 'CTYNAME', 'POPESTIMATE1999', 'POPESTIMATE2000', 'POPESTIMATE2001', 'POPESTIMATE2002', 'POPESTIMATE2003', 'POPESTIMATE2004', 'POPESTIMATE2005', 'POPESTIMATE2006', 'POPESTIMATE2007', 'POPESTIMATE2008', 'POPESTIMATE2009', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015', 'Note', 'CDC_WNV_Record', 'CDC_NI_Record', 'CDC_NNI_Record', 'NI_Total', 'Years_with_NI', 'NI_1999', 'NI_2000', 'NI_2001', 'NI_2002', 'NI_2003', 'NI_2004', 'NI_2005', 'NI_2006', 'NI_2007', 'NI_2008', 'NI_2009', 'NI_2010', 'NI_2011', 'NI_2012', 'NI_2013', 'NI_2014', 'NI_2015', 'NNI_Total', 'Years_with_NNI', 'NNI_1999', 'NNI_2000', 'NNI_2001', 'NNI_2002', 'NNI_2003', 'NNI_2004', 'NNI_2005', 'NNI_2006', 'NNI_2007', 'NNI_2008', 'NNI_2009', 'NNI_2010', 'NNI_2011', 'NNI_2012', 'NNI_2013', 'NNI_2014', 'NNI_2015', 'CDC_WNV_note', 'NI_IR_1999', 'NI_IR_20

In [69]:
#Retaining only Urban counties
wnv = wnv[wnv['Select_County'] == 1]
print(wnv.shape, "\n")

(852, 153) 



In [70]:
#This function is to create a dataframe for each "component" from NARR data files
#NARR data files = Temperature, Humidity and Precipitation

def add_data(wnv, narr_type, pre):
    '''
    add_data: Function to create a dataframe for each NARR component 
    wnv: Dataframe containing west nile virus data
    narr_type: It takes the values "air.sfc", "apcp" and "rhum.2m" which are the folder names of the specific data
    pre: The prefix for the column names
    return: A dataframe with each row corresponding to one county and each column corresponding to one month in a specific year
    '''

    narr_file = narr_type
    
    base = "narr_urban_county_data/"
    geoids = wnv.GEOID10.unique() #List of all counties 
    
    path = base + narr_file
    all_years = glob.glob(path + "/*.csv") #List of all individual monthly files
    
    df_list = [] 
    temp_county_df = pd.DataFrame(columns=["geoid"]) 

    for geo in geoids: #For all counties
    
        i = 1
        temp_county_df = pd.DataFrame({'geoid':[geo]}) #Create a dataframe for the county
    
        for filename in all_years: #For all individual monthly files
 
            temp_df = pd.read_csv(filename, index_col=None, header=0)
            temp_df = temp_df [temp_df["GEOID"] == geo]
            temp_df = temp_df [["GEOID", "NAME", "min", "max", "mean"]]

            year = filename[-11:-7] #Capturing year from input file
            month = filename[-6:-4] #Capturing month from input file
            col_head = pre + "_" + year + "_" + month #Creating the column name based on year and month
            mean = temp_df['mean'].iloc[0] 
        
            temp_county_df.insert(i, col_head, mean) #Adding columns to county dataframe
            i = i+1
    
        df_list.append(temp_county_df) #Appending every county data to a list
    
    data = pd.concat(df_list, axis=0, ignore_index=True) #Merging all counties data into one dataframe
    
    return data

In [72]:
#Creating the temperature dataframe

start2 = time.process_time() #To calculate the time taken to read 204 files
temperature = add_data(wnv, 'air.sfc', 'temp')
print("Temperature Dataframe created in ", time.process_time() - start2, "seconds")
print("Shape of the Temperature Dataframe: ", temperature.shape)
temperature.head(2)

Temperature Dataframe created in  1675.1880319999982 seconds
Shape of the Temperature Dataframe:  (852, 205)


,geoid,temp_2005_04,temp_2005_10,temp_2012_08,temp_2015_01,temp_2007_01,temp_2000_08,temp_2009_10,temp_2009_04,temp_2009_05,...,temp_2010_01,temp_2000_10,temp_2000_04,temp_2009_08,temp_2009_09,temp_2000_05,temp_2000_11,temp_2012_11,temp_2012_05,temp_2005_09
0,1001,289.581116,291.530701,301.059753,280.201111,282.598145,303.537048,291.368896,290.723694,296.050842,...,277.802582,293.259644,290.032837,300.633423,298.278137,297.588440,285.571320,285.007996,297.622589,298.562408
1,1003,292.088806,294.914673,301.561605,286.371724,287.003052,303.901306,295.728719,292.648397,297.185852,...,283.698690,295.833700,293.208862,301.574259,299.819621,298.811605,290.364787,289.731913,298.842448,300.400675


In [73]:
#Creating the precipitation dataframe

start3 = time.process_time() #To calculate the time taken to read 204 files
precipitation = add_data(wnv, 'apcp', 'prec')
print("Precipitation Dataframe created in ", time.process_time() - start3, "seconds")
print("Shape of the Precipitation Dataframe: ", precipitation.shape)
precipitation.head(2)

Precipitation Dataframe created in  1639.3089760000003 seconds
Shape of the Precipitation Dataframe:  (852, 205)


,geoid,prec_2005_04,prec_2005_10,prec_2012_08,prec_2015_01,prec_2007_01,prec_2000_08,prec_2009_10,prec_2009_04,prec_2009_05,...,prec_2010_01,prec_2000_10,prec_2000_04,prec_2009_08,prec_2009_09,prec_2000_05,prec_2000_11,prec_2012_11,prec_2012_05,prec_2005_09
0,1001,5.668302,0.753516,4.248878,3.371792,2.909578,1.600176,4.870292,2.846911,6.538771,...,4.705506,0.896645,3.477417,3.327592,6.671547,1.177589,6.507411,0.898148,4.458558,2.315675
1,1003,8.950225,0.262567,7.796551,3.108120,2.712963,2.176495,4.096749,2.793004,5.538939,...,5.071959,0.631503,1.716327,4.541365,5.659177,0.740194,6.142436,0.839033,4.393895,3.389329


In [74]:
#Creating the humidity dataframe

start4 = time.process_time() #To calculate the time taken to read 204 files
humidity = add_data(wnv, 'rhum.2m', 'humidity')
print("Humidity Dataframe created in ", time.process_time() - start4, "seconds")
print("Shape of the Humidity Dataframe: ", humidity.shape)
humidity.head(2)

Humidity Dataframe created in  1698.6453739999997 seconds
Shape of the Humidity Dataframe:  (852, 205)


,geoid,humidity_2005_04,humidity_2005_10,humidity_2012_08,humidity_2015_01,humidity_2007_01,humidity_2000_08,humidity_2009_10,humidity_2009_04,humidity_2009_05,...,humidity_2010_01,humidity_2000_10,humidity_2000_04,humidity_2009_08,humidity_2009_09,humidity_2000_05,humidity_2000_11,humidity_2012_11,humidity_2012_05,humidity_2005_09
0,1001,75.112236,72.799408,74.057236,76.640259,76.05545,62.012241,82.579178,76.129517,81.253365,...,76.489143,55.624550,73.271858,76.993057,80.407364,71.453308,72.858421,71.935555,70.351341,73.884529
1,1003,76.923716,75.017761,78.907557,79.591787,80.07194,71.517426,82.470418,79.402883,83.760890,...,78.871048,64.529165,73.547658,77.856049,80.854782,76.897639,75.170639,76.702372,76.237757,77.400441


In [76]:
#Merging the temperature, humidity and precipitation dataframes based on geoid

narr_df = pd.merge(pd.merge(temperature, humidity, on='geoid'), precipitation, on='geoid')
narr_df.shape

(852, 613)

In [77]:
# Adding the weather data to the WNV data
final = wnv.merge(narr_df, left_on='GEOID10', right_on='geoid')
final.shape

(852, 766)

In [79]:
# Adding the "State" column to add the climate zone column
final['ansi'] = final['GEOID10'].astype(str).str[:2] 
final['ansi'] = final['ansi'].astype(int)
final['state'] = final['ansi'].map(ansi_dict)
final['climate_zone'] = final['state'].map(climate_dict2) 
final.shape

(852, 769)